# Розділ 7: Створення чат-додатків
## Швидкий старт з API моделей Github

Цей ноутбук адаптовано з [репозиторію прикладів Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), який включає ноутбуки для доступу до сервісів [Azure OpenAI](notebook-azure-openai.ipynb).

# Огляд  
"Великі мовні моделі - це функції, які перетворюють текст на текст. Отримавши вхідний рядок тексту, велика мовна модель намагається передбачити текст, який ітиме далі"(1). Цей "швидкий старт" познайомить користувачів з концепціями LLM високого рівня, основними вимогами пакетів для початку роботи з AML, м'яким введенням у дизайн промптів та кількома короткими прикладами різних випадків використання.

## Зміст  

[Огляд](#огляд)  
[Як використовувати сервіс OpenAI](#як-використовувати-сервіс-openai)  
[1. Створення вашого сервісу OpenAI](#1.-створення-вашого-сервісу-openai)  
[2. Встановлення](#2.-встановлення)    
[3. Облікові дані](#3.-облікові-дані)  

[Випадки використання](#випадки-використання)    
[1. Підсумовування тексту](#1.-підсумовування-тексту)  
[2. Класифікація тексту](#2.-класифікація-тексту)  
[3. Генерація нових назв продуктів](#3.-генерація-нових-назв-продуктів)  
[4. Тонке налаштування класифікатора](#4.тонке-налаштування-класифікатора)  

[Посилання](#посилання)

### Побудуйте свій перший промпт  
Ця коротка вправа надасть базове введення для подання промптів моделі в Github Models для простого завдання "підсумовування".

**Кроки**:  
1. Встановіть бібліотеку `azure-ai-inference` у своєму середовищі Python, якщо ви ще цього не зробили.  
2. Завантажте стандартні допоміжні бібліотеки та налаштуйте облікові дані для Github Models.  
3. Виберіть модель для вашого завдання  
4. Створіть простий промпт для моделі  
5. Надішліть запит до API моделі!

### 1. Встановіть `azure-ai-inference`

In [1]:
%pip install azure-ai-inference

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 2. Імпортуйте допоміжні бібліотеки та створіть облікові дані

In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. Пошук правильної моделі  
Моделі GPT-3.5-turbo або GPT-4 можуть розуміти та генерувати природну мову.

In [3]:
# Виберіть модель загального призначення curie для тексту
model_name = "gpt-4o"

## 4. Дизайн промпту  

"Магія великих мовних моделей полягає в тому, що, навчаючись мінімізувати цю похибку передбачення на великих обсягах тексту, моделі в результаті вивчають концепції, корисні для цих передбачень. Наприклад, вони вивчають такі концепції як"(1):

* як правильно писати
* як працює граматика
* як перефразовувати
* як відповідати на запитання
* як вести розмову
* як писати багатьма мовами
* як кодувати
* тощо.

#### Як керувати великою мовною моделлю  
"З усіх входів до великої мовної моделі, безумовно, найбільш впливовим є текстовий промпт"(1).

Великі мовні моделі можна спонукати до створення виводу кількома способами:

Інструкція: Скажіть моделі, що ви хочете
Завершення: Спонукайте модель завершити початок того, що ви хочете
Демонстрація: Покажіть моделі, що ви хочете, за допомогою:
Кількох прикладів у промпті
Багатьох сотень або тисяч прикладів у навчальному наборі даних для тонкого налаштування"

#### Існують три основні рекомендації щодо створення промптів:

**Показуйте та розповідайте**. Чітко вказуйте, що ви хочете, через інструкції, приклади або їх комбінацію. Якщо ви хочете, щоб модель розташувала список елементів в алфавітному порядку або класифікувала абзац за настроєм, покажіть їй, що саме ви хочете.

**Надавайте якісні дані**. Якщо ви намагаєтеся створити класифікатор або змусити модель слідувати певній схемі, переконайтеся, що є достатньо прикладів. Обов'язково перевірте свої приклади — модель зазвичай достатньо розумна, щоб зрозуміти основні орфографічні помилки та дати вам відповідь, але вона також може припустити, що це навмисно, і це може вплинути на відповідь.

**Перевіряйте налаштування**. Параметри temperature та top_p контролюють, наскільки детермінованою є модель при генерації відповіді. Якщо ви просите відповідь, де є лише одна правильна відповідь, то вам варто встановити їх нижче. Якщо ви шукаєте більш різноманітні відповіді, то можливо вам захочеться встановити їх вище. Найпоширенішою помилкою, яку люди роблять з цими налаштуваннями, є припущення, що вони є елементами керування "розумністю" або "креативністю".

Джерело: https://github.com/Azure/OpenAI/blob/main/How%20to/Completions.md

### 5. Надсилаємо!

In [4]:
# Створіть свій перший промпт
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

'The use of the Oxford comma (also referred to as the serial comma) is a matter of style, and whether you should always use it depends on the context and the style guide you’re following. The Oxford comma is the comma placed before the conjunction (typically "and" or "or") in a list of three or more items—for example:\n\n**With Oxford comma:** "I love my parents, Oprah, and Elon Musk."  \n**Without Oxford comma:** "I love my parents, Oprah and Elon Musk."\n\n### When the Oxford Comma Should Be Used:\n1. **Clarity:** In some cases, omitting the Oxford comma can lead to ambiguity or unintended meanings. For example, the second example above could imply that your parents are Oprah and Elon Musk. Including the Oxford comma avoids this confusion.\n2. **Style Guides That Recommend It:** Certain style guides, like **The Chicago Manual of Style** and **Oxford University Press**, generally recommend its use for clarity and consistency.\n3. **Consistency in Writing:** If you are working on a doc

### Повторіть той самий виклик, як порівнюються результати?

In [5]:
response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

'The use of the Oxford comma (also called the serial comma) is a stylistic choice, and whether or not it "should" always be used depends largely on context, the style guide you\'re following, and personal or organizational preference. Here\'s a breakdown to help you decide:\n\n### What is the Oxford Comma?\nThe Oxford comma is the comma placed before the final item in a list, right before the conjunction ("and" or "or"). For example:\n- With Oxford comma: I brought apples, oranges, and bananas.\n- Without Oxford comma: I brought apples, oranges and bananas.\n\n### Reasons to Always Use the Oxford Comma\n1. **Clarity**: The Oxford comma can prevent ambiguity and clarify meaning, especially in complex sentences. For example:\n   - Without Oxford comma: I dedicate this book to my parents, Oprah Winfrey and God.\n     (This implies your parents are Oprah Winfrey and God.)\n   - With Oxford comma: I dedicate this book to my parents, Oprah Winfrey, and God.\n     (This clearly lists three se

## Підсумовування тексту  
#### Завдання  
Підсумуйте текст, додавши 'tl;dr:' в кінці текстового уривку. Зверніть увагу, як модель розуміє, як виконувати низку завдань без додаткових інструкцій. Ви можете експериментувати з більш описовими промптами, ніж tl;dr, щоб модифікувати поведінку моделі та налаштувати підсумок, який ви отримуєте(3).  

Останні роботи продемонстрували значні успіхи в багатьох завданнях та тестах NLP шляхом попереднього навчання на великому корпусі тексту з подальшим тонким налаштуванням на конкретне завдання. Хоча зазвичай архітектура не залежить від завдання, цей метод все ще вимагає наборів даних для тонкого налаштування, специфічних для завдання, які містять тисячі або десятки тисяч прикладів. На відміну від цього, люди зазвичай можуть виконувати нове мовне завдання лише з кількох прикладів або з простих інструкцій - щось, з чим поточні системи NLP все ще значною мірою борються. Тут ми показуємо, що масштабування мовних моделей значно покращує ефективність, не залежну від завдання, з кількома прикладами, іноді навіть досягаючи конкурентоспроможності з попередніми найсучаснішими підходами тонкого налаштування.

Tl;dr

# Вправи для кількох випадків використання  
1. Підсумовування тексту  
2. Класифікація тексту  
3. Генерація нових назв продуктів

In [6]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"

In [7]:
#Встановлення кількох додаткових типових параметрів під час виклику API

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

'Scaling up language models significantly enhances task-agnostic few-shot performance in NLP tasks, reducing reliance on extensive fine-tuning datasets and achieving competitiveness with state-of-the-art fine-tuned models.'

## Класифікація тексту  
#### Завдання  
Класифікуйте елементи на категорії, надані під час виведення. У наступному прикладі ми надаємо як категорії, так і текст для класифікації у промпті(*playground_reference). 

Запит клієнта: Привіт, одна з клавіш на клавіатурі мого ноутбука нещодавно зламалася, і мені потрібна заміна:

Класифікована категорія:

In [8]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]

inquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:

Classified category:


In [9]:
#Встановлення кількох додаткових типових параметрів під час виклику API

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

'**Classified category:** Hardware Support'

## Генерація нових назв продуктів
#### Завдання
Створіть назви продуктів з прикладів слів. Тут ми включаємо в промпт інформацію про продукт, для якого ми збираємося генерувати назви. Ми також надаємо схожий приклад, щоб показати схему, яку ми хочемо отримати. Ми також встановили високе значення температури, щоб збільшити випадковість та отримати більш інноваційні відповіді.

Опис продукту: Домашній міксер для молочних коктейлів
Ключові слова: швидкий, здоровий, компактний.
Назви продуктів: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Опис продукту: Пара взуття, яка може підходити для будь-якого розміру стопи.
Ключові слова: адаптивний, що підходить, omni-fit.

In [10]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

Product description: A home milkshake maker
Seed words: fast, healthy, compact.
Product names: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Product description: A pair of shoes that can fit any foot size.
Seed words: adaptable, fit, omni-fit.


In [11]:
#Встановлення кількох додаткових типових параметрів під час виклику API

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

"**Product Names for Shoes**:  \n1. Adaptix  \n2. FitAll  \n3. OmniStep  \n4. FlexiFit  \n5. OneSize  \n6. StepSync  \n7. ShapeShift   \n8. UniversalSole  \n\nLet me know if you'd like more options or refinements! 😊  "

## Індивідуальне завдання


> Створіть чат-додаток, який класифікує помилки компіляції коду.

In [15]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

# 1. Ініціалізація клієнта всередині клітинки для уникнення NameError
def get_ai_client():
    auth_token = os.environ.get("GITHUB_TOKEN")
    if not auth_token:
        raise ValueError("GITHUB_TOKEN не знайдено в змінних оточення")
    
    return ChatCompletionsClient(
        endpoint="https://models.inference.ai.azure.com",
        credential=AzureKeyCredential(auth_token),
    )

# 2. Функція аналізу помилок
def analyze_compilation_error(error_log):
    # Створюємо клієнта безпосередньо в момент виклику або використовуємо глобального
    try:
        client = get_ai_client()
        
        system_instruction = (
            "Ти — старший інженер-програміст. Твоя задача — проаналізувати помилку компіляції.\n"
            "Структура відповіді:\n"
            "1. Мова програмування.\n"
            "2. Тип помилки (напр. SyntaxError, ReferenceError).\n"
            "3. Пояснення причини простими словами.\n"
            "4. Код після виправлення."
        )
        
        response = client.complete(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": error_log}
            ],
            temperature=0.2
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Помилка при запиті до ШІ: {e}"

# 3. Тестовий запуск
test_error = "Java error: System.otl.println('Hello');"
print(f"Вхідна помилка: {test_error}")
print("-" * 40)

# Виклик функції
result = analyze_compilation_error(test_error)
print(result)

Вхідна помилка: Java error: System.otl.println('Hello');
----------------------------------------


1. **Мова програмування**: Java  
2. **Тип помилки**: Compilation Error (likely a `cannot find symbol` error)  
3. **Пояснення причини простими словами**: У коді є помилка в назві методу. У Java правильний метод для виведення тексту на консоль — `System.out.println()`. У вашому коді написано `System.otl.println()`, що є помилкою через неправильне написання `out` як `otl`.  

4. **Код після виправлення**:  
```java
System.out.println("Hello");
```
